In [76]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
from nltk.corpus import wordnet
import nltk

In [56]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
import pickle


In [3]:
data= pd.read_csv("raw data.csv")

In [4]:
data.head()

,Doc_name,Category,Content
0,001.txt,Business,Ad sales boost Time Warner profit\n\nQuarterly...
1,002.txt,Business,Dollar gains on Greenspan speech\n\nThe dollar...
2,003.txt,Business,Yukos unit buyer faces loan claim\n\nThe owner...
3,004.txt,Business,High fuel prices hit BA's profits\n\nBritish A...
4,005.txt,Business,Pernod takeover talk lifts Domecq\n\nShares in...


In [27]:
data = data.drop(["Content_parsed", "Doc_name", "Category"], axis = 1)
data.head()

,Content,Category_codes
0,Ad sales boost Time Warner profit Quarterly p...,0
1,Dollar gains on Greenspan speech The dollar h...,0
2,Yukos unit buyer faces loan claim The owners ...,0
3,High fuel prices hit BA's profits British Air...,0
4,Pernod takeover talk lifts Domecq Shares in U...,0


## Feature Engineering

### 1. Convert Everything To Lowercase

In [6]:
#Convert everything to lowercase
data["Content_parsed_1"] = data["Content"].str.lower()

In [8]:
data.loc[0, "Content_parsed_1"]

'ad sales boost time warner profit\n\nquarterly profits at us media giant timewarner jumped 76% to $1.13bn (â£600m) for the three months to december, from $639m year-earlier.\n\nthe firm, which is now one of the biggest investors in google, benefited from sales of high-speed internet connections and higher advert sales. timewarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. its profits were buoyed by one-off gains which offset a profit dip at warner bros, and less users for aol.\n\ntime warner said on friday that it now owns 8% of search-engine google. but its own internet business, aol, had has mixed fortunes. it lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. however, the company said aol\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to si

### 2. Handling Possesive Pronouns

In [9]:
#Possesive Pronouns
data["Content_parsed_2"] = data["Content_parsed_1"].str.replace("'s", "").str.replace("’s", "")

In [11]:
data["Content_parsed_2"][0]

'ad sales boost time warner profit\n\nquarterly profits at us media giant timewarner jumped 76% to $1.13bn (â£600m) for the three months to december, from $639m year-earlier.\n\nthe firm, which is now one of the biggest investors in google, benefited from sales of high-speed internet connections and higher advert sales. timewarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. its profits were buoyed by one-off gains which offset a profit dip at warner bros, and less users for aol.\n\ntime warner said on friday that it now owns 8% of search-engine google. but its own internet business, aol, had has mixed fortunes. it lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. however, the company said aol underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign 

### 2. Removing Stop Words

In [12]:
#List of stop words extracted from https://gist.github.com/sebleier/554280
stop_words = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]

In [13]:
#Removing stop words
data["Content_parsed_3"] = data['Content_parsed_2']
data["Content_parsed_3"] = data["Content_parsed_3"].str.replace("’", "'")
for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    data['Content_parsed_3'] = data['Content_parsed_3'].str.replace(regex_stopword, '')

In [14]:
data['Content_parsed_3'][0]

'ad sales boost time warner profit\n\nquarterly profits   media giant timewarner jumped 76%  $1.13bn (â£600m)    months  december,  $639m year-earlier.\n\n firm,       biggest investors  google, benefited  sales  high-speed internet connections  higher advert sales. timewarner  fourth quarter sales rose 2%  $11.1bn  $10.9bn.  profits  buoyed  - gains  offset  profit dip  warner bros,   users  aol.\n\ntime warner   friday    owns 8%  search-engine google.    internet business, aol,   mixed fortunes.  lost 464,000 subscribers   fourth quarter profits  lower    preceding  quarters. ,  company  aol underlying profit  exceptional items rose 8%     stronger internet advertising revenues.  hopes  increase subscribers  offering  online service free  timewarner internet customers     sign  aol existing customers  high-speed broadband. timewarner    restate 2000  2003    probe    securities exchange commission (),   close  concluding.\n\ntime warner fourth quarter profits     analysts\' expectat

### 2. Removing Punctuation Signs

In [15]:
#Removing punctuation signs
data["Content_parsed_4"] = data["Content_parsed_3"]
punct_signs = [ "'", '"' , '“' , '”' , "\n" , "(" , ")" , "," , "." , "?", "-", ":"]
for sign in punct_signs:
    data["Content_parsed_4"] = data["Content_parsed_4"].str.replace(sign, " ")


In [16]:
data["Content_parsed_4"][0]

'ad sales boost time warner profit  quarterly profits   media giant timewarner jumped 76%  $1 13bn  â£600m     months  december   $639m year earlier    firm        biggest investors  google  benefited  sales  high speed internet connections  higher advert sales  timewarner  fourth quarter sales rose 2%  $11 1bn  $10 9bn   profits  buoyed    gains  offset  profit dip  warner bros    users  aol   time warner   friday    owns 8%  search engine google     internet business  aol    mixed fortunes   lost 464 000 subscribers   fourth quarter profits  lower    preceding  quarters     company  aol underlying profit  exceptional items rose 8%     stronger internet advertising revenues   hopes  increase subscribers  offering  online service free  timewarner internet customers     sign  aol existing customers  high speed broadband  timewarner    restate 2000  2003    probe    securities exchange commission       close  concluding   time warner fourth quarter profits     analysts  expectations    f

### 5. Lemmatization

In [20]:
def get_wordnet_pos(word):
    #Map POS tag to first character lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
             "N": wordnet.NOUN,
             "V": wordnet.VERB,
             "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [23]:
#Lemmitization
lemmatizer = WordNetLemmatizer()
all_articles = data["Content_parsed_4"]
all_articles_l = []
for article in all_articles:
    article_words = article.split()
    article_words_l = []
    for article_word in article_words:
        article_words_l.append(lemmatizer.lemmatize(article_word, pos = get_wordnet_pos(article_word)))
    article_l = " ".join(article_words_l)
    all_articles_l.append(article_l)
data["Content_parsed_5"] = all_articles_l

In [24]:
data["Content_parsed_5"][0]

'ad sale boost time warner profit quarterly profit medium giant timewarner jumped 76% $1 13bn â£600m month december $639m year earlier firm big investor google benefit sale high speed internet connection high advert sale timewarner fourth quarter sale rise 2% $11 1bn $10 9bn profit buoyed gain offset profit dip warner bros user aol time warner friday own 8% search engine google internet business aol mixed fortune lose 464 000 subscriber fourth quarter profit low precede quarter company aol underlie profit exceptional item rise 8% strong internet advertising revenue hope increase subscriber offering online service free timewarner internet customer sign aol exist customer high speed broadband timewarner restate 2000 2003 probe security exchange commission close conclude time warner fourth quarter profit analyst expectation film division profit slump 27% $284m help box office flop alexander catwoman sharp contrast year earlier final film lord ring trilogy boost full year timewarner post p

This looks good now!

In [25]:
data.head()

,Doc_name,Category,Content,Content_parsed_1,Content_parsed_2,Content_parsed_3,Content_parsed_4,Content_parsed_5
0,001.txt,Business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit quarterly p...,ad sale boost time warner profit quarterly pro...
1,002.txt,Business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gains on greenspan speech\n\nthe dollar...,dollar gains on greenspan speech\n\nthe dollar...,dollar gains greenspan speech\n\n dollar hit...,dollar gains greenspan speech dollar hit ...,dollar gain greenspan speech dollar hit high l...
2,003.txt,Business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer faces loan claim\n\nthe owner...,yukos unit buyer faces loan claim\n\nthe owner...,yukos unit buyer faces loan claim\n\n owners ...,yukos unit buyer faces loan claim owners em...,yukos unit buyer face loan claim owner embattl...
3,004.txt,Business,High fuel prices hit BA's profits\n\nBritish A...,high fuel prices hit ba's profits\n\nbritish a...,high fuel prices hit ba profits\n\nbritish air...,high fuel prices hit ba profits\n\nbritish air...,high fuel prices hit ba profits british airwa...,high fuel price hit ba profit british airway b...
4,005.txt,Business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lifts domecq\n\nshares in...,pernod takeover talk lifts domecq\n\nshares in...,pernod takeover talk lifts domecq\n\nshares u...,pernod takeover talk lifts domecq shares uk ...,pernod takeover talk lift domecq share uk drin...


### 6. Label Coding

In [26]:
category_codes = {"Business" : 0,
                 "Entertainment" : 1,
                 "Politics" : 2,
                 "Sport": 3,
                 "Tech": 4}

In [27]:
data["Category_codes"] = data.Category

In [28]:
data.Category_codes = data.Category_codes.replace(category_codes)

In [29]:
data.tail()

,Doc_name,Category,Content,Content_parsed_1,Content_parsed_2,Content_parsed_3,Content_parsed_4,Content_parsed_5,Category_codes
2221,397.txt,Tech,BT program to beat dialler scams\n\nBT is intr...,bt program to beat dialler scams\n\nbt is intr...,bt program to beat dialler scams\n\nbt is intr...,bt program beat dialler scams\n\nbt introduc...,bt program beat dialler scams bt introducin...,bt program beat dialler scam bt introduce init...,4
2222,398.txt,Tech,Spam e-mails tempt net shoppers\n\nComputer us...,spam e-mails tempt net shoppers\n\ncomputer us...,spam e-mails tempt net shoppers\n\ncomputer us...,spam -mails tempt net shoppers\n\ncomputer use...,spam mails tempt net shoppers computer users...,spam mail tempt net shopper computer user cont...,4
2223,399.txt,Tech,Be careful how you code\n\nA new European dire...,be careful how you code\n\na new european dire...,be careful how you code\n\na new european dire...,careful code\n\n european directive soft...,careful code european directive softwa...,careful code european directive software write...,4
2224,400.txt,Tech,US cyber security chief resigns\n\nThe man mak...,us cyber security chief resigns\n\nthe man mak...,us cyber security chief resigns\n\nthe man mak...,cyber security chief resigns\n\n man making ...,cyber security chief resigns man making c...,cyber security chief resigns man make computer...,4
2225,401.txt,Tech,Losing yourself in online gaming\n\nOnline rol...,losing yourself in online gaming\n\nonline rol...,losing yourself in online gaming\n\nonline rol...,losing online gaming\n\nonline role playing ...,losing online gaming online role playing ga...,lose online game online role play game time co...,4


In [30]:
columns_to_hold = ['Doc_name', 'Category', 'Content', 'Content_parsed_5', 'Category_codes']
data = data[columns_to_hold].rename(columns = {"Content_parsed_5" : "Content_parsed"})

In [31]:
data.head()

,Doc_name,Category,Content,Content_parsed,Category_codes
0,001.txt,Business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sale boost time warner profit quarterly pro...,0
1,002.txt,Business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech dollar hit high l...,0
2,003.txt,Business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim owner embattl...,0
3,004.txt,Business,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profit british airway b...,0
4,005.txt,Business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq share uk drin...,0


### 6. Train-Test Split   
We will keep the test size as only 0.15. Dring training, we will also use cross-validation to get more realistic accuracy.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data["Content_parsed"], data["Category_codes"], 
                                                    test_size=0.25, random_state=34)

In [38]:
print("""X_train shape: {0}    y_train shape: {1}
X_test shape: {2}      y_test shape: {3}""".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

X_train shape: (1669,)    y_train shape: (1669,)
X_test shape: (557,)      y_test shape: (557,)


In [39]:
print("y_train value count:\n" ,y_train.value_counts())
print("y_test value count:\n", y_test.value_counts())

y_train value count:
 0    395
3    373
2    318
4    304
1    279
Name: Category_codes, dtype: int64
y_test value count:
 3    138
0    116
1    107
2     99
4     97
Name: Category_codes, dtype: int64


Almost equal distribution of all categories. Hence **balanced** training and testing datasets.

### 7. Text Representation using TFIDF 


TF-IDF parameters used:
1. ngram_range = (1, 2)    
    We want to capture unigrams and bigrams to capture important words as well as words that appear together
2. Minimum df = 10   
    To get rid of words that appear extremely rarely in less than 10 documents
3. Maximum df = 1.   
    No limit to this as this more frequently occuring words will help in classification
4. max_features = 300   
    Considering top 300 features for building the vocabulary dictionary. 

In [44]:
ngram_range = (1, 2)
min_df = 10
max_df = 1.
max_features = 300

In [45]:
tfidf2 = TfidfVectorizer(lowercase=False, 
                         stop_words=None, 
                         ngram_range=ngram_range, 
                         max_df=max_df, 
                         min_df=min_df,
                        max_features= 300)

In [46]:
tfidf2.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=300,
                min_df=10, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [47]:
features_train = tfidf2.transform(X_train)
labels_train = y_train

In [48]:
features_test = tfidf2.transform(X_test)
labels_test = y_test

### Performing Chi-squared test on Features

In [53]:
category_dict = {"Business" : 0,
                "Entertainment" : 1,
                "Politics" : 2,
                "Sport" : 3,
                "Tech" : 4}

for category, category_id in category_dict.items():
    chi2_features = chi2(features_train, labels_train == category_id)
    indices = np.argsort(chi2_features[0])
    feature_names = np.array(tfidf2.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split()) == 1]
    bigrams = [v for v in feature_names if len(v.split()) == 2]
    print("Category = " + category + "\n")
    print("5 most dependent unigrams: " + str(unigrams[-7:]))
    print("5 most dependent bigrams: " + str(bigrams[-2:])+ "\n")

Category = Business

5 most dependent unigrams: ['firm', 'company', 'share', 'economy', 'bank', 'growth', 'oil']
5 most dependent bigrams: ['prime minister', 'told bbc']

Category = Entertainment

5 most dependent unigrams: ['music', 'song', 'band', 'actor', 'star', 'award', 'film']
5 most dependent bigrams: ['told bbc', 'prime minister']

Category = Politics

5 most dependent unigrams: ['lord', 'mp', 'election', 'blair', 'tory', 'party', 'labour']
5 most dependent bigrams: ['told bbc', 'prime minister']

Category = Sport

5 most dependent unigrams: ['player', 'england', 'win', 'injury', 'champion', 'match', 'cup']
5 most dependent bigrams: ['told bbc', 'prime minister']

Category = Tech

5 most dependent unigrams: ['phone', 'microsoft', 'technology', 'computer', 'mobile', 'user', 'software']
5 most dependent bigrams: ['told bbc', 'prime minister']



Observation : It is sensible to say that the unigrams captured corresponding to each category are apt.

In [58]:
with open("Pickles\\X_train.pickle", "wb") as file:
    pickle.dump(X_train, file)

with open("Pickles\\X_test.pickle", "wb") as file:
    pickle.dump(X_test, file)
    
with open("Pickles\\y_train.pickle", "wb") as file:
    pickle.dump(y_train, file)

with open("Pickles\\y_test.pickle", "wb") as file:
    pickle.dump(y_test, file)
    
with open("Pickles\\features_train.pickle", "wb") as file:
    pickle.dump(features_train, file)
    
with open("Pickles\\features_test.pickle", "wb") as file:
    pickle.dump(features_test, file)
    
with open("Pickles\\labels_train.pickle", "wb") as file:
    pickle.dump(labels_train, file)
    
with open("Pickles\\labels_test.pickle", "wb") as file:
    pickle.dump(labels_test, file)
    
with open("Pickles\\tfidf.pickle", "wb") as file:
    pickle.dump(tfidf2, file)
    
with open("Pickles\\data.pickle", "wb") as file:
    pickle.dump(data, file)